### Astronomical Source Detection using UDF  f105w image

#### Following  the  tutorial  found  at https://sep.readthedocs.io/en/v1.0.x/tutorial.html,  but  using  the astropy  fits  routines  instead  of fitsio

#### Importing libraries such as numpy astropy.io.fits instead of fitsio and matplotlib.pyplot <br> *(by Veronika C. Joseph)*

In [ ]:
import numpy as np
import numpy.ma as ma
import sep
import matplotlib.pyplot as plt
from astropy.io import fits 
from astropy import units
from astropy.utils.data import download_file
from matplotlib import rcParams
from matplotlib.colors import LogNorm
from astropy.table import Table
import matplotlib.colors as colors

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

#### Read in image from a FITS file usitng astropy.io fits.open method   <br> *(by Veronika C. Joseph)*

In [ ]:
# hdulist returns an object HDUList that is list like collection of HDU objects

# opening file with memmap = True since its big
fits_file = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
hdulist = fits.open(fits_file, memmap=True)

# -- below shows info about the file we are reading-in
print('Image info:\n')
hdulist.info()
print('\n')

# image_data stores primary info about our FITS file
image_data = hdulist[0].data 
print('\n')

# used below code to view parts of array to view data that we have from fits file
# and to try to locate how are bad values represented (such as values of noise)
# print(image_data[0:25, 0:25])
print('Original image data array slice:\n')
print(image_data[350:360, 2000:2010])
print(image_data[2000:2010, 1000:1010])
print('\n')

# *** Below was used to try to test masking some values in array to see what happens with our background image

# print('Image data array slice with values < 0 replaced by 0.0:\n')
# image_data[image_data < 0] = 0.0
# taking log of positive pixel values: 
# image_data[image_data > 0] = np.log(image_data[image_data > 0])
# print(image_data[1700:1710, 500:510])
# print('\n')

# boolean array, mark any element < 0 as true
#print('Boolean image data array slice for values < 0:\n')
#non_negative_image_data[image_data < 0] 
#print(non_negative_image_data[1700:1710, 500:510])

# print('TESTING MASKING')
# masked_image_data = ma.masked_less(image_data, 0)
# take log of positive pixel values
# log_image_data = np.log(masked_image_data)
# print('\nLog image data \n')
# print(log_image_data[1700:1705, 500:505])

# print(ma.masked_less(image_data, 0)[1700:1710, 500:510])
# print(masked_image_data[1700:1710, 500:510])

#mask = image_data > 0
#image_data[mask] = 777


#print(image_data.size)

# close fits file since we stored all the info into variable
hdulist.close()


#### Display the image from FITS file that was read in using plt.imshow method and plt.colorbar() <br> *(by Veronika C. Joseph)*

In [ ]:
# m - stores mean of our image_data 
# s - stores standard deviation of image_data 
# plt.imshow(..) displays an image data on 2D regular raster
#      cmap - is a colormap instance that maps scalar data to colors
#      vmin / vmax - defines the data range that the colormap covers 
#      origin - places [0,0] index of the array in the lower left corner of axes

m, s = np.mean(image_data), np.std(image_data)
# m, s = np.mean(masked_image_data), np.std(masked_image_data)
# using log of our positive image data 
# m, s = np.mean(masked_image_data), np.std(masked_image_data)


plt.imshow(image_data, interpolation='nearest', cmap='bone', vmin = m-s, vmax = m+s, origin='lower')
plt.colorbar();

# saving fits file without object detection marks
plt.savefig('f105w_image_from_fits_file.png')

##### Plotting using LogNorm to get better idea of objects on this image

In [ ]:
plt.imshow(image_data, cmap='RdBu_r', norm = LogNorm(), origin='lower')

cbar = plt.colorbar(ticks=[5.e3, 1.e3, 2.e4])
cbar.ax.set_yticklabels(['5,000', '10,000', '20,000'])

 #### Subtracting background using sep.bckground method, which will return Background object that holds the spatially varying background and spatially varying background noise level <br> *(by Veronika C. Joseph)*

In [ ]:
image_data = image_data.byteswap(inplace=True).newbyteorder()

bkg = sep.Background(image_data, bw=64, bh=64, fw=3, fh=3)

#### Obtaining various information about our image from Background object bkg by using SEP methods such as globalback, globalrms <br> *(by Veronika C. Joseph)*

In [ ]:
# get a "global" mean and noise of the image background:
print('Global background level: ', bkg.globalback)
print('Global background RMS:   ', bkg.globalrms)

In [ ]:
# evaluate background as 2-D array, same size as original image
bkg_image = bkg.back()

# bkg_image = np.array(bkg)   # equivalent to above

#### Showing our 2D background array using plt.imshow method specifying various attributes of graph  <br> *(by Veronika C. Joseph)*

In [ ]:
# show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray',origin='lower')
plt.colorbar();
plt.savefig('f105w_2D_background.png', bbox_inches="tight", dpi=600)

In [ ]:
# evaluate the background noise as 2-D array, same size as original image
bkg_rms = bkg.rms()

#### Similar process here but instead we are showing 2D background noise array  <br> *(by Veronika C. Joseph)*

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest',cmap='gray',origin='lower')
plt.colorbar();
plt.savefig('f105w_2D_background_noise.png', bbox_inches="tight", dpi=600)

##### Subtracting background from our image and saving the result in data_sub variable <br>*(by Veronika C. Joseph)*

In [ ]:
# subtract the background 
data_sub = image_data - bkg

#### Running object detection on the background-subtracted data using sep.extract method that extracts sources from an image by specifying various controlling detection tresholds <br> *(by Veronika C. Joseph)*

In [ ]:
# data_sub - is our new image data with subtracted background 
# 1.5 - is our current treshold pixel value for detection 
# err - specifies pixel-by-pixel detection treshold 

# testing various thresholds to try to eliminate noise 
objects = sep.extract(data_sub, 5, err=bkg.globalrms)

In [ ]:
# how many objects were detected
print('Number of objects detected: ', len(objects))

#### Using objects['x'] and objects['y'] we'll obtain the centroid coordinates of the objects that were detected above.  Using Ellipse imported from matplotlib.patches, we'll overplot the object coordinates with basic shape parameters to check for the detected objects on our FITS image using custom plotting attributes <br>*(by Veronika C. Joseph)*

In [ ]:
from matplotlib.patches import Ellipse
from matplotlib.colors import LogNorm

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='bone', vmin=m-s, vmax=m+s, origin='lower')
#plt.imshow(image_data, cmap='bone', norm=LogNorm())

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy = (objects['x'][i], objects['y'][i]),
               width  = 6*objects['a'][i],
               height = 6*objects['b'][i],
               angle  = objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

    
plt.savefig('f105w_with_circled_objects.png', bbox_inches="tight", dpi=600)

#### Next, we are performing simple circular aperture photometry with a 3.0 pixel radius at location of the objects using sep.sum_circle method that sums data in circular aperture(s) <br>*(by Veronika C. Joseph)*

In [ ]:
# flux, fluxerr, flag: are all 1-D array with one entry per object
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))  

#### Histogram of fluxes: 

### Extra Credit 3-image rgb file

#### Extra Credit: Download the f125w and f160w images of the HUDF at the same website, and make a 3-color false image of the UDF using RGB -> f160w, f125w, f105w. Save the image as a PNG. <br> (by Veronika C. Joseph)

##### Read in the 3 fits files that we'll use for creating RGB png image<br> (by Veronika C. Joseph)

In [ ]:
file_f160w_r = "hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits"
file_f125w_g = "hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits"
file_f105w_b = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"

f160w_r = fits.open(file_f160w_r)[0].data

f125w_g = fits.open(file_f125w_g)[0].data

f105w_b = fits.open(file_f105w_b)[0].data


#### Let's plot all three files using the default color map and by using log10  <br>(by Veronika C. Joseph)

In [ ]:
f160w_r_log = np.log10(f160w_r)
f_f160w_r = plt.figure(figsize=(7,7))
plt.imshow(f160w_r_log)

In [ ]:
f125w_g_log = np.log10(f125w_g)
f_f125w_g = plt.figure(figsize=(7,7))
plt.imshow(f125w_g_log)

In [ ]:
f105w_b_log = np.log10(f105w_b)
f_f105w_b = plt.figure(figsize=(7,7))
plt.imshow(f105w_b_log)

#### Making a 3-color image out of our 3 fits files: <br> (by Veronika C. Joseph)

In [ ]:
rgb_image = np.zeros((3600,3600,3))

rgb_image[:,:,0] = f105w_b
rgb_image[:,:,1] = f125w_g
rgb_image[:,:,2] = f160w_r

rgb_image = colors.hsv_to_rgb(rgb_image)

#### Plotting our RGB image generated from 3 image fits files <br> (by Veronika C. Joseph)

In [ ]:
f = plt.figure(figsize=(7,7))
plt.imshow(rgb_image)

#### Saving our RGB image as a png file <br> (by Veornika C. Joseph)

In [ ]:
plt.imsave("rgb_image.png", rgb_image)